# Deep Feedforward Network 深度前馈网络

> 也叫 feedforward neural networks/multilayer preceptrons 多层感知机, 这是一类重要的监督学习网络.

目标就是估计一些函数 $f^*(\boldsymbol{x};\boldsymbol{\theta})$, 模型学习参数 $\boldsymbol{\theta}$ 来得到最优函数估计( **万能函数近似器 Universal function approximation** )，e.g., 对于分类器，$y = f^*(\boldsymbol{x})$ 就是一个对输入 $\boldsymbol{x}$ 的映射，映射到类别 $y$.

**feedforward** 就是信息流顺序地从 $\boldsymbol{x}$ 经过 $f$ 中间计算再到输入 $\boldsymbol{y}$, 而 **feedback 反馈** 就是指模型的输出还会继续回到模型. 同时存在两者的网络称为 **recurrent neural network(复发神经网络, 用于生成序列 sequence processnig)**.

网络用有向无环图来描述每个函数之间的复合关系, 例如 $f(x) = f^{(3)}(f^{(2)}(f^{(1)}(x)))$, 这是个三层(layer, 又称此网络的深度 depth 为 3)网络, 其中 $f^{(1)}$ 称为第一层, 最后一层叫做输出层; 中间这些就是 **隐藏层 hidden layers**, 因为这些层得出的结果往往还不是想要的结果, 隐藏层中的函数称为 **激活函数 activation function**.

我们可以把网络层想象为一堆并行处理的单元组成的, 每一个神经元都是一个 **vector-to-scalar function**.

> It is best to think offeedforward networks as function approximation machines that are designed toachieve statistical generalization, occasionally drawing some insights from what weknow about the brain, rather than as models of brain function.

线性模型因为是线性函数, 没办法处理不同输入数据之间的交互关系, MLP 正好可以用于解决这样的限制. 其实对线性模型应用 **核方法(kernel trick)**, 达到非线性变换的目的.

不过 kernel function 的选择:

* 使用一个非常泛化的核函数, 比如高斯核(RBF kernel) 就是映射到一个无限维度空间, 不过过于泛化的话会导致测试结果很糟糕
* 手动为每个任务找到最优的核函数, 在深度学习之前这几乎是主流, 不过这种方法的统一性不足, 而且需要花很多的功夫.
* 深度学习其实可以说就是用于训练学习得到这个核函数, 不过深度学习没有怎么考虑到问题的凸优化方面(而 SVM/核方法就很注重这个方面), 这是一种短板. 深度学习在可以从一个很宽泛的函数家族来进行训练的基础上, 还可以使用人为的先验知识, 并且先验知识不用像方法 2 那样去选择一个具体的函数, 只需要提供一个偏好的函数家族就行.

> 其实使用核函数可以理解一种表示学习方法, 将原始表示映射为其他表示.

MLP 学习的是从 $x$ 到 $y$ 的 **确定性映射(deterministic mappings)**, 如果引入 feedback, 模型学习的可以是 **stochastic mappings 随机映射**, 并且还可以得出单个向量的概率分布.

除了跟原来线性模型一样需要设计的 **优化方法 optimizer**, **损失函数 cost function** 还有 **输出单元的格式** 之外, MLP 还需要设计 **网络的架构** (e.g., 层数, 各层之间连接性, 还有每层的单元). 深度神经网络需要计算复杂函数的梯度, 到时候我们将使用 **back-propagation 反向传播算法** 及其现代泛化形式用来高效的计算梯度.

## Example: Learning XOR

原来神经网络提出时就因为没办法实现 XOR 陷入了长达几年的瓶颈.

我们可以用原来的最小化 MSE (虽然实际应用中, MSE 不是一个合适的二元数据的损失函数)的线性回归方法来拟合这样的一个定义域只有四个离散值的三维函数, 然而我们会发现拟合出来的函数不好用, 一个解决方案是用类似于核技巧的方法去转换特征空间.

> 使用线性模型的时候, 当 $x_1 = 0$ 的时候, 模型输出随着 $x_2$ 增大而增大, 然而当 $x_1 = 1$ 的时候, 模型输出又需要随着 $x_2$ 增大而减小, 所以用一个固定的在 $x_2$ 上的权重没办法做到.

我们可以使用神经网络来对表示进行非线性转换。大部分神经网络(的隐藏层）都是使用参数控制的 **仿射变换（affine transformation，可以是非线性变换）**，然后再使用一个固定的非线性函数作为 **激活函数（activation function）**。 前馈神经网络中默认推荐使用 **ReLU（整流线性单元 rectified linear function）** $g(z) = z_+ = max\{0, z\}$. 这样一般的激活函数就是 $\boldsymbol{h} = g(\boldsymbol{W}^\top \boldsymbol{x} + \boldsymbol{x})$.

> affine 就是在 linear 的基础上加了一个 偏置 biases, 如果 biases 为 $0$ 那就是一个 linear

> 使用 `ReLu` 的好处: 采用sigmoid等函数，算激活函数时（指数运算），计算量大，反向传播求误差梯度时，求导涉及除法，计算量相对大，而采用 ReLU 激活函数，整个过程的计算量节省很多

对于 XOR 问题，我们使用一层隐藏层，并且输出层就使用线性变换（i.e., $\boldsymbol{w}^\top \boldsymbol{h} + b$，其中 $h$ 为隐藏层的输出）. 这样整个网络的函数表达式就是 $f(\boldsymbol{x}; \boldsymbol{W}, \boldsymbol{c}, \boldsymbol{w}, b) = \boldsymbol{w}^\top \max \{ 0, \boldsymbol{W}^\top \boldsymbol{x} + \boldsymbol{c}\} + b$

输入为

![微信截图_20181024154258.png](./images/微信截图_20181024154258.png)


略过中间训练过程，我们得到

![](./images/微信截图_20181024154733.png)


这样隐藏层就完成了一个非线性转换.

## gradient-based learning

NN(neural network) 的学习跟机器学习经典的 linear model 并没有太大区别, 都是指定优化过程, 损失函数, 还有模型家族. 最大的区别在于, NN 的 nonlinearity 导致其损失函数 loss function/cost function 没办法应用凸优化技巧了. **随机梯度优化 Stochastic gradient descent** 用于没有这么好的收敛性的非凸函数, 但是 **对初始值很敏感** (像凸函数的话, 任意初始值都能收敛到最优值, 虽然实践中可能会遇到 **数值问题 numerical problems** ). NN 的优化总的来说还是 **iterative gradient-based optimization**, 不过 NN 的函数的梯度计算往往比经典的机器学习算法更加复杂, 所以后面将提到的 `反向传播算法 back-propagation`  及其现代泛化形式能够依然能够高效准确地计算出梯度.


### Stochastic Gradient Descent (SGD)

对于机器学习的训练, 很多时候会因为训练集很大(为了取得很好地泛化)导致其计算量也会很大. ML 的 cost function 一般是 **sum of some per-example loss function**, 比如使用每个样本的 **NLL, negative condition log-likelihood 负对数似然** 的和作为整个训练集的损失函数:

$$
J(\theta) = \mathbb{E}_{x, y \sim \hat{p}_{\text{data}}} L(x, y, \theta) = \frac{1}{m} \sum_{i=1}^m L(x^{(i)}, y^{(i)}, \theta)
$$

> 写成标量形式只是为了简单书写, 一般都是矢量情况. 其中单个样本的损失函数就是其 NLL: $-\log p(y|x; \theta)$

![NLL](./images/NLL.png)

`SGD` 的核心思想就是把梯度看作是一个数学期望, 并且 **这个期望能够用一小部分样本来估计**. 也就是说, 在算法的每一步, 我们可以从样本集中取样一小部分作为 **minibatch**: $\mathbb{B} = \{ x^{(1)}, \cdots, x^{(m^\prime)}\}$, minibatch 的大小一般是 **1 ~几百**, 而且在训练集增加的时候, minibatch 的大小 **一般不变**(这样对于一个固定的模型大小, 每次 SGD 更新所花费的跟整个训练集的大小无关, 因为 minibatch 的大小是固定的) . 所以估计的梯度就是

$$
g = \frac{1}{m^\prime} \nabla_\theta \sum_{i=1}^{m^\prime} L(x^{(i)}, y^{(i)}, \theta)
$$

所以这之后参数的学习就是:

$$
\boldsymbol{\theta} = \boldsymbol{\theta} - \epsilon \boldsymbol{g}
$$

> 其中标量 $\epsilon$ 就是学习率

梯度下降这种方法应用在非凸优化问题在过去常常被认为是蛮干, 不过在后面深度学习的应用中, 我们可以看到虽然没办法在有意义的时间内到达局部最优点, 但是 **能够快速找到用起来效果不错的很低的损失值**. 当数据集增大的时候, 收敛所需要的更新次数也往往在增加, 不过就算训练集规模是无限, SGD 也能在访问有限个样本的时候取得一个可能最好的 test error, SDG 对比普通的梯度下降的渐进复杂度差不多就是 $O(1)$ 和 $O(m)$ 的区别.

> 历史小故事. 在 DL 出现之前, ML 常用 kernel trick + linear model 的组合来训练得到 nonlinear model. 但是很多的 kernel learning algorithms 需要构建一个 $m \times m$ 矩阵 $G_{i, j} = k(x^{(i)}, x^{(j)})$, 这个计算量是 $O(m^2)$, 对于百万数据样本的情况, 这样的计算量是不可忍受的.

## cost function

NN 的 cost function 和像前面说的 linear model 这种 parametric models 差不多, 一样是定义一个概率分布 $p(y|x; \theta)$, 然后应用最大似然(i.e., 训练集和模型预测之间的 **交叉熵** ).甚至可以简单地只预测 $y$ 在 $x$ 上的一些 **统计量 statistic**. 和线性模型一样的, 我们也可以加上 **正则项 regularization** (这里是用于处理概率密度/质量异常的大/小的情况).

### Conditional probability(cross entropy)

对 cost function 还有一个要求就是其梯度要足够大和可预测(predictable)来引导学习算法. 如果函数过于 **饱和 saturate**(也就是说太拼 **平 flat** 了), 其梯度就会很小, 这种情况经常在隐藏层和输出层出现(e.g. 激活函数中含有指数函数, 这会导致当参数是一个很小的负数的时候, 函数饱和), 用 **NLL 负对数似然** 可以帮助缓解这种情况. 

### Learning Conditional Statistics

我们可以认为是去学习选择整个函数而不只是去学习其参数. 这时候 cost function 就是一个 **functional** 而不是一个函数, 这个 **functional** 是一个从函数到数值的映射. 解决这种优化问题需要用到 **变分法 calculus of variations**.

变分法主要用于导出下面两种优化问题的结果:

* $f^* = \arg \min_f \mathbb{E}_{x, y \sim p_{\text{data}}} \lVert y - f(x) \rVert^2$ (就是 MSE) 得到 $f^*(x) = \mathbb{E}_{y \sim p_{\text{data}} (y | x)} [ y ]$ (证明过程还不知道, 得到的这个函数是对每个 $x$ 对应的 $y$ 的均值的预测, 不知道原理)
* $f^* = \arg \min_f \mathbb{E}_{x, y \sim p_{\text{data}}} \lVert y - f(x) \rVert_1$ (使用 L1 范数, 得到的函数是对每个 $x$ 对应的 $y$ 的 **中位数 median** 的预测, 不懂原理), 这种 cost function 叫做 **mean absolute error**.

> 在使用基于梯度优化 MSE/mean absolute error 的时候, 往往会得到不太好的结果. 所以还是用交叉熵比较多一点.

## output units

> NN(不仅仅是 Deep Learning 的, 而是表示学习范畴的)一般都需要用到 output unit

cost function 和 output units 的选择一般是紧紧地耦合的, 因为 cost function 一般用的是交叉熵. 不过 output unit 一般也可以用作为 hidden unit.

隐藏层学习出来的新表示(设为 $\boldsymbol{h}$)还需要通过输出层进行一些额外的变换.

### linear unit for Gaussian ouput distributions

用线性的仿射变换 $\hat{\boldsymbol{y}} = \boldsymbol{W}^\top \boldsymbol{h} + \boldsymbol{b}$, 线性仿射出来的结果常用作条件高斯的均值:

$$p(\boldsymbol{y} | \boldsymbol{x}) = \mathcal{N} (\boldsymbol{y}; \hat{\boldsymbol{y}}, \mathbb{I})$$

然后再用 NLL 就可以把这个条件高斯作为 cost function 了.

> 这里面的 covariance 默认用单位矩阵, 因为 NLL 后精度矩阵就是一个固定的常数量. 不过 covariance 也可以用 MLE(最大似然估计)来估计, 不过 covariance 必须是正定矩阵的要求导致不太好用 linear output layer, 可以用其他类型的 output unit 来完成 covariance 的参数估计.

### sigmoid units for Bernoulli output distri.(abbr of Distributions)

有些任务的输出是二值的, 例如二分类器.

NN 只需要学习 $P(y = 1 | \boldsymbol{x})$, 因为是离散分布, 所以要确保其概率质量在 $[0, 1]$, 如果用 linear unit 和阈值(threshold):

$$P(y = 1 | \boldsymbol{x}) = \max \{0, \min \{1, \boldsymbol{w}^\top \boldsymbol{h} + b\}\}$$

但是上述表达式不好用梯度方法进行优化, 因为在 $[0,1]$ 之外梯度变成了 0, 没办法给学习算法提供导向了. 所以我们使用 sigmoid 函数在达到这个目的

$$\hat{y} = \sigma \left(\boldsymbol{w}^\top \boldsymbol{h} + b\right)$$

为了导出上述公式, 我们可以先假定一个 $y$ 上的非归一化(unnormalized)也就是(概率密度累加之和不为 1)的 Bernoulli 分布($z = \boldsymbol{w}^\top \boldsymbol{h} + b$), $\tilde{P}(y = 1) = e^z, \tilde{P}(y = 0) = e^0 = 1$, 然后我们在归一化(也就是都除以 $\tilde{P}(0) + \tilde{P}(1) = 1 + e^z$), 归一化之后:

$$
P(y = 1) = \frac{1}{1 + e^{-z}} \\
P(y = 0)  = \frac{1}{1 + e^z}
$$

最后可以使用一个简单的映射 $2y - 1, ( 0 \rightarrow -1, 1 \rightarrow 1)$ 把上述两个公式统一为 $P(y) = \sigma \left( \left(2y - 1\right) z\right)$

所以 cost function 可以写作(使用 NLL)

$$
\begin{eqnarray} 
J(\boldsymbol{\theta}) &=& - \log P(y | \boldsymbol{x}) \\
&=& - \log \left( \left( 2y - 1 \right) z \right) \\
&=& \zeta \left( \left( 1 - 2y \right) z \right)
\end{eqnarray} 
$$

通过最终转换成 softplus, 我们发现只有在 $y = 1, z \rightarrow + \infty$ 或 $y = 1, z \rightarrow - \infty$ (这两种情况都是正确的情况) softplus 才会趋近于饱和(saturate), 在错误情况下, softplus 能够提供很好地梯度信息.

> 使用 MSE 之类的没有使用 $\log$ 去消除 $\exp$ 的方法作为 loss function 的话, 使用 sigmoid 还是会导致过于饱和, 进而找不到有效的梯度信息

> sigmoid 因为在整个实数定义域范围内的值域为 $(0, 1)$, 跟概率范围 $[0,1]$ 的范围有点出入. 所以在软件实现上, 为了避免 sigmoid 出现数值问题(下溢为 0), 应该把 NLL 写作 $z$ 的函数而不是关于 $\sigma (z))$ 的函数.

### softmax units for multinoulli output distri.

对于多值离散变量的分布, 我们一般都会使用 `softmax`, `softmax` 可以看做是一种泛化形式的 `sigmoid`. `softmax` 常用作分类器的输出层, 也可以用作模型内部(隐藏层), 用于从多个内部变量的选项中选择一个.

类似于 sigmoid 的思路, 我们先定义一个线性仿射 $\boldsymbol{z} = \boldsymbol{W}^\top \boldsymbol{h} + \boldsymbol{b}$, $z_i = \log \tilde{P}(y = i | \boldsymbol{x})$. softmax 定义为

$$\text{softmax} (\boldsymbol{z})_i = \frac{\exp(z_i)}{\sum_j \exp(z_j)}$$

NLL 作为 cost function 就是 $- \sum_i \log P(y = i; \boldsymbol{z}) = - \sum_i \log \text{softmax} (\boldsymbol{z})_i = - \sum_i \left( z_i - \log \sum_j \exp(z_j) \right)$

我们希望 $\log \sum_j \exp(z_j) \approx \max_j z_j$ 尽可能小, 所以对于这个 $\max_j z_j$, 因为其值最大, 说明其分类是最准确的, 但是又因为减去了这个 $\log \sum_j \exp (z_j)$, 所以这个最准确的样本却对整个训练集的 cost 之和的贡献很小, 不如其他不准确的样本产生的 cost 值的影响大.

softmax 的一个很重要的性质就是 $\text{softmax} (z)  = \text{softmax}(z + c)$ ( $c$ is an constant scalar), 使用这一个性质, $\text{softmax}(z) = \text{softmax}(z - \max_i z_i)$ 可以得到一个更加数值稳定的版本的 softmax(这也表明了 softmax 就是为了衡量输入 $\boldsymbol{z}$ 中各维度上的值和 $\max_i z_i$ 的偏离情况).

softmax 在输入中有一个维度上的值特别大或者特别小于 $max_i z_i$ 的时候会饱和.

> 其实按照 softmax 的要求, $\boldsymbol{z}$ 的前 $n - 1$ 项的和就等于最后一项(你多我少的情况有点像神经元的抑制作用), 所以最后一项是可以省略的(i.e., 冗余的, 这叫做 overparametrizes), 不过省略还是不省略只是学习过程中有点差别, 描述的分布集都是一样的, 而且 overparametrizes 实现起来更加简单一点.

> softmax 更加像 softened(连续, 可微 differential) 版本的 $\arg \max$, softened 版本的 $\max$ 应该是 $\text{softmax}(\boldsymbol{z})^\top \boldsymbol{z}$

### other output units 

一般为了适应 MLE 的 $\log$, 我们设计 cost function 都是使用 $\exp$.

> 一般来说, NN 不是直接学习出预测 $y$ 的函数, 而是一个 $y$ 上分布的参数的有关函数 
> $\boldsymbol{\omega}(\boldsymbol{x}) = f(\mathtt{x}; \theta)$ (因为一般分布都是提前设置好的, 学习算法主要就是学习其参数).

前面我们讲过的条件高斯作为分布的情况, 我们都是直接忽略了精度矩阵/协方差矩阵的学习, 而是简单的设为单位阵. 现在我们需要学习协方差的话, 跟学习均值是一样的, 都是由参数函数 $\boldsymbol{\omega}(\boldsymbol{x})$ 得到, 一般是学习 **(对角 diagonal)精度矩阵** 而不是协方差, 协方差会随着输入值的不同在学习过程中变化, 这种模型叫做 **异方差 heteroscedastic** 模型.

使用对角精度矩阵的好处:

* 因为对数似然只需要用到精度矩阵对角线上的元素
* 在求梯度的时候, 用精度矩阵的话就可以不用除法或者平方了
* 在协方差矩阵/精度矩阵为正定这个限制上, 对角阵会比普通矩阵简单很多而且计算量小很多, 只要求每个元素的数值为正就行, 这样可以方便使用 softplus 函数

在处理 **多峰回归 multimodal regression** 的时候, 我们经常使用 **Gaussian mixture**, $n$ 个组件 component 的条件高斯定义为:

$$p(y | x) = \sum_{i=1}^n p(c = i | x)\mathcal{N}(y; \mu^{(i)}(x), \Sigma^{(i)}(x)))$$

有以下限制:

* component 的概率服从关于 **潜变量 latent variable 就是非直接从数据集中观测出来的变量** 的 multinoulli distri., 一般来说可以直接用 n-D 向量上的 softmax 函数.

> 有时候会因为某一个方差特别小导致出现数值不稳定情况, 比如特别小的方差被作除数用来计算梯度导致梯度爆炸的时候. 可以使用 **梯度修剪 clip gradient 就是设置一个阈值** 或者启发式 heuristically 地调整梯度.

## Hidden Units

> hidden unit 是 MLP 独有的, 并且现在这个领域还是挺活跃的, 因为导向性的理论原理还不多.

ReLU 就是一般 hidden unit 的默认选择, ReLU 不是处处可微的, 在 $0$ 这一个点就不可微, 不过在 DL 的实际应用中梯度下降依然表现良好, 这是因为 NN 训练算法一般不会到达 cost function 的局部最优点, 而只是函数值显著的下降. 而且, 虽然 ReLU 在 $0$ 这一点不可微, 其左右导数都是有定义的, 而且在软件实现上, 框架一般在遇到这个情况的时候是返回其中一边的单侧导数或者启发式的返回一个比较小下界为 $0$ 的值, 而不是返回 undefined 或者抛出异常, 反正用户不用太在意就好.

大部分隐藏层单元都可以描述为输入向量 $\boldsymbol{x}$, 然后做一个仿射变换 $\boldsymbol{z} = \boldsymbol{W}^\top \boldsymbol{x} + \boldsymbol{b}$, 然后在应用一个 element-wise 的非线性的 **激活函数** (一般是 ReLU). 大部分隐藏层单元的区别就在于这个激活函数的选择了.

> 当模型比较接近于线性(i.e., unit 或其他操作的行为比较接近线性), 可以取得更加好的性能

### ReLU (Rectified Linear Unit)

优点

* 跟 linear unit 很像, 计算简单
* 不管 unit 是否激活, ReLU 之后还能能够保留很大的梯度(0 也算大?)
* 二阶梯度几乎都为 $0$, 在激活的时候梯度为 $1$, 梯度的方向信息比一般的需要二阶梯度信息的激活函数要更加的有用

一些 good practice

* affine transformation 中的 $\boldsymbol{b}$ 一般取所有元素都是很小的正整数, e.g., $0.1$, 这样可以使大多数输入都能使 ReLU 处于激活状态

缺点

* 当 ReLU 的激励值为 0 的时候, 没办法使用基于梯度的优化算法. 所以有很多 ReLU 泛化版本就关注于能够处处有梯度, 不过这些泛化版本能少能够有更好的性能(毕竟 ReLu 已经够简单的了)

> 我现在发现把 activation function 翻译为 **激励函数** 或许更加好点,  这样可以跟激活状态区分开.

一些变种(泛化形式):

前三种泛化版本都是基于一个非 $0$ 坡度 $\alpha_i, h_i = g(\boldsymbol{z}, \boldsymbol{\alpha})_i = \max (0, z_i) + \alpha_i \min (0, z_i)$

* Absolute value rectification 将 $\alpha_i$ 固定为 $-1$, 这样激励函数就是绝对值函数了. 这个用于从图像中识别物体, 因为图像翻转也不会影响特征的获取, 所以这样做挺有道理的.
* **leaky ReLU** 把 $\alpha_i$ 设置为很小的值, e.g., $0.01$
* **parametric ReLU** 把 $\alpha_i$ 当做可学习的参数
* **Maxout unit** 比起前面的 element-wise 的 ReLU,  maxout 将 $\boldsymbol{z}$ 分为很多个有 $k$ 个元素的组, 然后再求每组的最大值作为输出: $g(\boldsymbol{z})_i  = \max_{j \in \mathbb{G}^{(i)}} z_j$, 这是一种 piece-wise linear function. 

Maxout 的优点

* 因为 maxout 这种分段函数可以用来(粗略的)拟合任意的凸函数, maxout 相当于学习激活函数而不是处理 units 之间的关系, 例如把分组设置为 $2$, 就可以学习成跟 ReLU, absolute value rectification, leaky 之类的函数. 
* 可以避免 **catastrophic forgetting 灾难性遗忘** 现象.

### logistic sigmoid and hyperbolic tangent

$$
g(z) = \sigma(z) \\
g(z) = \tanh(z)
$$

> 而且 $\tanh(z) = 2 \sigma (2z) - 1$

$\text{sigmoid}$ 一般不用再 feedward network, 因为他们比较容易饱和, 不利于梯度, 可以用 $\tanh$, $\tanh$ 在接近于 $0$ 跟 $f(x) = x$(identity function) 很相似.

### other

用的不多, 其实很多其他不流行的激活函数性能也不错, 不过没有什么特别大的提升, 所以业界不会有太大兴趣.

还有一种就是没有激励函数, 或者说用的就是 identity function $f(x) = x$, 这样只能构建出线性模型. 不使用激励函数可以减少参数的个数.

* softmax
* Radial basis function(RBF, 高斯) $h_i = \exp(-\frac{1}{\sigma^2}\lVert \boldsymbol{W}_{:, i} - \boldsymbol{x}\rVert^2)$, 很容易饱和, 不好优化
* softplus, 相当于 smooth 版本的 ReLU, 但是经验上表明这个不如 ReLU 好用
* hard tanh, 有点像 ReLU, $g(h) = \max(-1, \min(1, h))$

## Architecture Design

一般的 NN 都是每层是 group of units, 然后这些层又是 chain structure. 层数越多, 每层需要的单元数和参数会更加少, 并且测试集上的泛化效果会更加好, 但是会导致更加难优化(按照经验来说也不是越深越好). 但是层数不变的时候, 参数的增多对泛化效果不一定有正向效果, 可能会因为诸如过拟合之类的问题造成效果变差.

> 总的来说还是需要靠实验才能确定网络架构是否理想, 理论上比起 ML 更加玄学...

**universal approximation theorem** 表明线性输出层加至少一层由 挤压 squashing 激励函数组成的隐藏层组成的 feedforward network 能够近似出任意有限维空间上的 **Borel 可测函数(Borel measureable function, 可以理解为 $\mathbb{R}^n$ 的有界封闭子集上的连续函数)**. 并且后面的研究表明最坏情况下(只有一层隐藏层的 shallow network)最多需要 **指数个** 隐藏层单元.

![deeper network](./images/deeper_network.png)

> 上图直观地表明层数和隐藏层单元的指数关系

> 虽然 MLP 有能力表示大部分的函数, 不过不一定能够保证能够学习出来, 因为囿于优化算法的能力或者过拟合. 还有就是按照 **no free launch theorem**, 不存在一个通用的万能学习算法.

### Other

有些架构不一定会 chain connection, 有可能会有 **skip connnection**, 比如直接跳过中间几层直接连到后面的网络层去, 这样有利于从靠近输入层的位置去流动梯度.

还有就是两层之间如何连接, 一般是 **全连接 fully connected**, 也就是每一个输出单元都是由所有的输入单元连接着. 减少连接可以减少参数数量进而减少计算量. 例如, **CNN** 就是一种利用 **稀疏连接 sparse connection** 

## Back-propagation 反向传播

**forward propagation 正向传播**: 从输入 $\boldsymbol{x}$ 中获取初始信息然后从隐藏层一层一层向前传递进而产生输出 $\hat{\boldsymbol{y}}$

反向传播用于将信息反向流动用于计算梯度, 反向传播只是一种便于计算梯度的方法, 不是特定于 MLP 也不是表明 NN 的输出为反向.

$$\nabla_{\boldsymbol{x}} f(\boldsymbol{x}, \boldsymbol{y})$$ where $x$ is a set of variables whose derivatives are desired, and $y$ is an additional set of variables that are inputs to the function but whose derivatives are not required

> back-propagation 亦可用于多输出函数, 用于计算其 Jacobian. 不过 DL 领域下, 我们暂且先只考虑单输出函数.

computational graph language 用 **node** 表示一个变量(scalar/vector/matrix/tensor, 也可以是 **symbol**, 在计算图的时候全部用 symbol 抽象化, 然后再在用的时候把数值带进去, 像 TensorFlow 和 Theano 都是这样做的); **operation(or, gate)** 是一个或多个变量的简单函数, 这里不是一般性地(without loss of generality)限定其输出是一个变量, 用 **edge** 来表示.

> 是否使用 symbol 可以将 backprop 分为 **number-to-number**, **symbol-to-number** 和 **symbol-to-symbol** 三种

**backprop** 建立在微积分链式法则的基础之上. 对于 $\boldsymbol{x} \in \mathbb{R}^m, \boldsymbol{y} \in \mathbb{R}^n, g: \mathbb{R}^m \rightarrow \mathbb{R}^n, f: \mathbb{R}^n \rightarrow \mathbb{R}$(实际 DL 训练中, 一般都是 tensor 而不是 vector), 链式法则描述为:

$$\nabla_\boldsymbol{x} z = \left( \frac{\partial \boldsymbol{y}}{\partial \boldsymbol{x}} \right)^\top \nabla_{\boldsymbol{y} } z$$

这就是一个 $n \times m$ 的 Jacobian 矩阵 乘以一个梯度 $\nabla_{\boldsymbol{y}} z$.

![](./images/computation_graph.png)

很多时候我们会遇到 **operation** 重复执行多次的计算, 有时候我们如果把计算结果存储下来(例如下图的 $f(w)$ )可以节省很多不必要的重复计算, 不过这会增加内存消耗.

![](./images/store.png)

一个 backprop 的简单例子

![](./images/backprop.png)

backprop 就是从输出层往后计算每一部分的导数(计算过程比较简单), 进而求出整体的导数.

backprop 就相当于原计算图中构建出一个部分导数和一些惩乘法和加法操作的新图, 可以认为这是一个线性行为. tensor 形式也只不过是 vector/matrix 形式的一个 **重整 rearrangement**. 在权衡是 store 存储这些重复的 subexpression 还是 re-compute 用于权衡计算量和内存占用之间的优化. backprop 在模块化求梯度过程可以解决重复 subexpression 造成的 **指数爆炸 exponential explosion** 问题.

不是一般性地, 这里用输出为一个标量, 输入为 $n_i$ 个元素的向量的简单情况来描述 backprop

首先构建计算图 computation graph $\mathcal{G}$

![comp_graph_algo](./images/comp_graph_algo.png)

然后再进行反向计算输出元素对所有隐藏层和输入层元素的偏导数并且存在表 $\text{grad_table}$ 中.

![](./images/backprop_algo.png)

更加泛化的定义为

对于 node 上的 variable $\mathsf{\boldsymbol{V}}$ (一个 tensor), 定义一下几个操作

* $\text{get_operation}(\mathsf{\boldsymbol{V}})$ 获取计算得到该变量的 operation
* $\text{get_consumers}(\mathsf{\boldsymbol{V}}, \mathcal{G})$ 获取 node 的所有子结点
* $\text{get_inputs}(\mathsf{\boldsymbol{V}}, \mathcal{G})$ 获取 node 的所有父节点

backprop 的现代形式(tensor 形式), 其实总的来说就是应用 chain rule, 不过进行的都是 tensor/matrix 运算.

![](./images/backprop_tensor.png)

build_grad:

![](./images/backprop_tensor_subfunction.png)


> 假定每个 operation 都是常数时间, 正向传播需要 $O(n)$ 的操作(也就是所有 $n$ 个 nodes), 而反向传播需要 $O(n^2)$ 的操作(也就是有向无环图最多所有 $O(n^2)$ 条边, 而且一般实际情况下更多是)

像 backprop 这种存储中间结果并且复用的思想成为 **动态规划 dynamic programming**

### Backprop in Fully Connected MLP

使用 **正向传播 forward propagation** 构建一个典型的 DNN(deep neural network) 的  computation graph, 其中 cost function 添加了正则项, $\theta$ 包含所有的参数($\boldsymbol{W}, \boldsymbol{b}$)

![](./images/fully_connected_MLP.png)

然后再使用 backprop 计算 cost function 对于 input 和 parameters 的梯度

![](./images/backprop_MLP.png)

> `w.r.t.` 是 **with respect to** 的缩写, 可翻译作 **对于**

### other derivative and differentiation techs.

也可以用与 backprop相反的 chain rule 方向来计算 subexpressions, 哪一个顺序能够话费 lower computational cost 是一个 **NP-complete** 问题. 而且像 backprop 这种 **automatic differential** (而且是 **reverse mode accumulation** ) 工具并不能像人类一样智能地处理一些显而易见的优化技巧, TensorFlow 之类的框架使用启发式(也就是写好的模式规则)来做优化.

当输出比输入要多很多的时候, 使用与 backprop 方向相反的 **forward mode accumulation**.

e.g.

对于矩阵乘法 $\boldsymbol{A B C D}$

如果 D 只有一列(i.e., vector), 而 A 有很多行, 这样比较适合像 backprop 一样反向地计算乘法, 而如果是相反的话, 用 **forward mode** 更加好点.

**Higher-Order derivatives**

有时候我们还需要知道二阶偏导矩阵 Hessian matrix, 不过常用 **Krylov methods**, 一种只用 matrix-vector 乘法的迭代近似计算的算法, 不过我们只能求出 Hessian 乘以任意向量得到的向量:

$$\boldsymbol{Hv} = \nabla_\boldsymbol{x} \left[ \left( \nabla_\boldsymbol{x} f(\boldsymbol{x}) \right)^\top \boldsymbol{v} \right]$$

一般这里面的向量 $\boldsymbol{v}$ 我们使用 **on-hot vector**(只有一个元素是 $1$ 其他都是 $0$ 的向量)

## 总结

现代 MLP 的核心思想基本上跟 1980 年代的差不多, 这几十年的突破大致在于:

* 使用 cross-entropy 取代 MSE 能够在使用 sigmoid 和 softmax 作为 output unit 的 MLP 上取得性能突破
* 用 piece-wise linear function(e.g. ReLU, 这东西后面还发现可以跟生物神经网络扯上关系...) 取代隐藏层的 sigmoid
* 数据集的增大, 计算性能的提高, 模型复杂度的增大

原来学术界对 feedforward neural network 的妖魔化一波三折...